In [ ]:
# Setup
!pip install pandas
!pip install google-cloud-storage
!pip install functions-framework==3.*
!pip install google-cloud-secret-manager
!pip install duckdb==1.1.0
!pip isntall gcsfs
!pip install pyarrow
!pip install openai

import os
os.environ['OPENAI_API_KEY'] ='KEY HERE'


# **Stock Overview**

Load parquet files to motherduck

In [ ]:
# imports
import functions_framework
from google.cloud import secretmanager
from google.cloud import storage
import json
import duckdb
import pandas as pd
from openai import OpenAI
import os
# setup
project_id = 'ba882-435919'
secret_id = 'motherduck_access_token'
version_id = '1'

# db setup
db = 'stocks'
reporting_db_schema = f"{db}.report"
created_tables = {}

############################################################### main task

# instantiate the services
sm = secretmanager.SecretManagerServiceClient()
storage_client = storage.Client()

# Build the resource name of the secret version
name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

# Access the secret version
response = sm.access_secret_version(request={"name": name})
md_token = response.payload.data.decode("UTF-8")

# initiate the MotherDuck connection through an access token through
md = duckdb.connect(f'md:?motherduck_token={md_token}')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ tbl: news

# read in from md
tbl_name = 'news_story_unique'
fetch = f"SELECT * FROM {reporting_db_schema}.{tbl_name};"
df = md.sql(fetch).df()


for i, row in enumerate(df.values):
  text = row[-3]
  summary = row[-1]
  if len(text) > 10: # and summary == 'Not Found'
    # send openai call
    # Summarizing
    prompt = f"""You are a helpful summarizer. You are given a news article, and will respond in this format:

    Summary: A 100 words or less summary of the article
    Sentiment: The general sentiment of the article with respect to the company being discussed (Positive, Neutral or Negative)
    Market Impact: A short note on potential impact of this news on stock price

    Article:{text}


    Article: {text}"""
    client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"{prompt}",
            }
        ],
        model="gpt-4o",
    )
    summary = chat_completion.choices[0].message.content
    print(summary)
    df.iloc[i,-1] = summary



# load from df
tbl_name = 'news_summarized'
reporting_sql = f"""
CREATE TABLE IF NOT EXISTS {reporting_db_schema}.{tbl_name} AS SELECT * FROM df;"""
print(f"{reporting_sql}")
md.sql(reporting_sql)
# del parquet_df




CREATE TABLE IF NOT EXISTS stocks.report.news_summarized AS SELECT * FROM df;
